### Connection to database and getting data

In [0]:
driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

database_host = "stocksdbserver.database.windows.net"
database_port = "1433" # update if you use a non-default port
database_name = "stocksdb"
table = "stocks"
user = "qangelot"
password = "mypassword1234"
url = f"jdbc:sqlserver://{database_host}:{database_port};database={database_name}"



In [0]:
df = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)


### Exploratory data analysis

In [0]:
# fixing date_ type
from pyspark.sql.functions import to_date

df = df.withColumn("date_",to_date("date_"))
df.show(5)


+----------+------------------+------------------+------------------+------------------+----------+------------------+----------+
 date_| high_| low_| open_| close_| volume| adj_close|stock_name|
+----------+------------------+------------------+------------------+------------------+----------+------------------+----------+
2017-01-03|29.082500457763672|28.690000534057617|28.950000762939453|29.037500381469727|1.151276E8|27.277639389038086| APPLE|
2017-01-03| 789.6300048828125| 775.7999877929688| 778.8099975585938| 786.1400146484375| 1657300.0| 786.1400146484375| GOOGLE|
2017-01-03|117.83999633789062|115.51000213623047|116.02999877929688|116.86000061035156| 2.06639E7|116.86000061035156| FACEBOOK|
2017-01-03|44.066001892089844| 42.19200134277344| 42.97200012207031| 43.39799880981445| 2.96165E7| 43.39799880981445| TESLA|
2019-04-18| 66.0| 60.32099914550781| 65.0| 62.0| 2.57647E7| 62.0| ZOOM|
+----------+------------------+------------------+------------------+------------------+----------+------------------+----------+
only showing top 5 rows

#### 1. Get return rate

In [0]:
from datetime import datetime
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, lit


def get_return_rate(df, stock, start_date, end_date): 
    '''Get the get_return_rate for specific stock and time interval'''

    temp = df[df["stock_name"] == str(stock)] 

    # convert string to date object
    d1 = datetime.strptime(start_date, "%Y-%m-%d")
    d2 = datetime.strptime(end_date, "%Y-%m-%d")

    # difference between dates in timedelta
    delta = d2 - d1

    # get init and end val
    init = temp.filter(temp.date_ == lit(start_date).cast('date')).collect()[0]['close_']
    end = temp.filter(temp.date_ == lit(end_date).cast('date')).collect()[0]['close_']

    # calculated by subtracting the initial value of the investment 
    # from its current value, and then dividing it by the initial value.
    result = ((end - init) / init * 100.00 )/delta.days

    return result


res = get_return_rate(df, 'AMAZON', '2017-01-03', '2017-02-03')
res

Out[4]: 0.24195594537491885

#### 2. Get moving average

In [0]:
from pyspark.sql.functions import avg
from pyspark.sql.window import Window
import pyspark.sql.functions as f


def get_moving_average(df, stock, start_date, end_date, num_points=5): 
    '''Function that take as input a dataframe, a stock name, a start_date, 
    an end_date, the number of points to consider for the moving average and 
    add a new column to the dataframe with the values of calculated moving average'''

    temp = df[df["stock_name"] == str(stock)] 

    # create a lag feature 
    window = Window.partitionBy("stock_name").orderBy(col("date_").asc())

    column = 'open_'
    for i in range(1, num_points+1):
        temp = temp.withColumn(f'prev_{str(column)}_{i}', 
                               f.lag(temp[str(column)], offset=i).over(window))

    # avg over the prev_price_ columns 
    prev_price_cols = temp.select(temp.colRegex(f"`prev_{column}_\d+`")).columns
    prev_price_cols = [col(c) for c in prev_price_cols]

    averageFunc = sum(x for x in prev_price_cols)/len(prev_price_cols)

    # create a moving_average_ column to store the result
    result = temp.withColumn(f'moving_average_{i}', (averageFunc))

    return result

res = get_moving_average(df, 'AMAZON', '2017-01-03', '2017-02-03', 5)
res.show()


+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
 date_| high_| low_| open_| close_| volume| adj_close|stock_name| prev_open__1| prev_open__2| prev_open__3| prev_open__4| prev_open__5| moving_average_5|
+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100.0|753.6699829101562| AMAZON| null| null| null| null| null| null|
2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500.0|757.1799926757812| AMAZON|757.9199829101562| null| null| null| null| null|
2017-01-05|782.4000244140625| 760.260009765625|761.5499877929688|780.4500122070312|5830100.0|780.4500122070312| AMAZON|758.3900146484375|757.9199829101562| null| null| null| null|
2017-01-06|799.4400024414062| 778.47998046875|782.3599853515625| 795.989990234375|5986200.0| 795.989990234375| AMAZON|761.5499877929688|758.3900146484375|757.9199829101562| null| null| null|
2017-01-09| 801.77001953125| 791.77001953125| 798.0|796.9199829101562|3446100.0|796.9199829101562| AMAZON|782.3599853515625|761.5499877929688|758.3900146484375|757.9199829101562| null| null|
2017-01-10| 798.0|789.5399780273438|796.5999755859375|795.9000244140625|2558400.0|795.9000244140625| AMAZON| 798.0|782.3599853515625|761.5499877929688|758.3900146484375|757.9199829101562| 771.643994140625|
2017-01-11| 799.5| 789.510009765625|793.6599731445312| 799.02001953125|2992800.0| 799.02001953125| AMAZON|796.5999755859375| 798.0|782.3599853515625|761.5499877929688|758.3900146484375|779.3799926757813|
2017-01-12|814.1300048828125| 799.5|800.3099975585938|813.6400146484375|4873900.0|813.6400146484375| AMAZON|793.6599731445312|796.5999755859375| 798.0|782.3599853515625|761.5499877929688| 786.433984375|
2017-01-13|821.6500244140625|811.4000244140625|814.3200073242188|817.1400146484375|3791900.0|817.1400146484375| AMAZON|800.3099975585938|793.6599731445312|796.5999755859375| 798.0|782.3599853515625| 794.185986328125|
2017-01-17| 816.0|803.4400024414062|815.7000122070312| 809.719970703125|3670500.0| 809.719970703125| AMAZON|814.3200073242188|800.3099975585938|793.6599731445312|796.5999755859375| 798.0|800.5779907226563|
2017-01-18| 811.72998046875| 804.27001953125| 809.5| 807.47998046875|2354200.0| 807.47998046875| AMAZON|815.7000122070312|814.3200073242188|800.3099975585938|793.6599731445312|796.5999755859375|804.1179931640625|
2017-01-19| 813.510009765625|807.3200073242188| 810.0|809.0399780273438|2540800.0|809.0399780273438| AMAZON| 809.5|815.7000122070312|814.3200073242188|800.3099975585938|793.6599731445312| 806.697998046875|
2017-01-20| 816.02001953125| 806.260009765625| 815.280029296875|808.3300170898438|3376200.0|808.3300170898438| AMAZON| 810.0| 809.5|815.7000122070312|814.3200073242188|800.3099975585938|809.9660034179688|
2017-01-23| 818.5|805.0800170898438|806.7999877929688|817.8800048828125|2797500.0|817.8800048828125| AMAZON| 815.280029296875| 810.0| 809.5|815.7000122070312|814.3200073242188| 812.960009765625|
2017-01-24| 823.989990234375| 814.5| 822.0|822.4400024414062|2971700.0|822.4400024414062| AMAZON|806.7999877929688| 815.280029296875| 810.0| 809.5|815.7000122070312| 811.456005859375|
2017-01-25|837.4199829101562|825.2899780273438|825.7899780273438| 836.52001953125|3922600.0| 836.52001953125| AMAZON| 822.0|806.7999877929688| 815.280029296875| 810.0| 809.5|812.7160034179688|
2017-01-26|843.8400268554688| 833.0| 835.530029296875|839.1500244140625|3586300.0|839.1500244140625| AMAZON|825.7899780273438| 822.0|806.7999877929688| 815.280029296875| 810.0|815.9739990234375|
2017-01-27|839.7000122070312|829.4400024414062| 839.0| 835.7700195312

In [0]:
res.write.format("jdbc") \
  .option("url", url) \
  .option("dbtable", "MovingAvgDf") \
  .option("user", user) \
  .option("password", password) \
  .save()